In [2]:
#MEDPC rat gambling task functions imports, will print "I am being executed!" if functional
import rgt_functions as rgt

#main imports 
import os
import pandas as pd
import numpy as np

# plotting imports 
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# stats imports 
import scipy.stats as stats

#the following lines prevents pandas from giving unecessary errors and increases the number of rows you can view 
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows',100)

I am being executed!


In [5]:
# file_names = ['..\\PhD_data\\BH06\\BH06_raw_round1-infusions.xlsx', '..\\PhD_data\\BH06\\BH06_raw_round1-makeup.xlsx'] 
file_names = ['BH06_raw_round1-infusions.xlsx', 'BH06_raw_round1-makeup.xlsx'] 

df = rgt.load_data(file_names)

#for plotting: 
title = '5-HT2c Antagonist' 

#lowest group number
startdose =  1
#highest group number
enddose = 4

In [6]:
df2 = rgt.edit_groups(df, orig_group = [0], new_group = [3], subs = [5])

df2 = rgt.drop_subjects(df, subs = [7])

df2 = rgt.get_choices(df)

#this is to simplify P1-P4 to designate each trial as "risky choice" (== 1 in column) or "optimal choice"
df2 = df2.loc[df2.option != 0]
df2['risky_choice'] = np.where(df2['option']>2,1,0)
df2.reset_index(drop = True, inplace = True)
df2

,MSN,StartDate,StartTime,Subject,Group,Box,Experiment,Comment,Session,Trial,...,Pun_Dur,Premature_Resp,Premature_Hole,Rew_Persev_H1,Rew_Persev_H2,Rew_Persev_H3,Rew_Persev_H4,Rew_Persev_H5,option,risky_choice
0,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,1.0,...,30,0,0,0,0,0,0,0,3,1
1,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,2.0,...,30,0,0,0,0,0,0,0,3,1
2,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,3.0,...,30,0,0,0,0,0,0,0,3,1
3,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,4.0,...,0,0,0,0,0,0,0,0,3,1
4,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,5.0,...,30,0,0,0,0,0,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4051,rGT_A-cue,03/19/20,13:48:44,9,4,1,0,NaN,78,44.0,...,30,0,0,0,0,0,0,0,3,1
4052,rGT_A-cue,03/19/20,13:48:44,9,4,1,0,NaN,78,45.0,...,0,0,0,0,0,0,1,0,2,0
4053,rGT_A-cue,03/19/20,13:48:44,9,4,1,0,NaN,78,46.0,...,0,0,0,0,0,0,0,0,3,1
4054,rGT_A-cue,03/19/20,13:48:44,9,4,1,0,NaN,78,47.0,...,0,0,0,0,0,0,0,0,3,1


In [7]:
def get_win_stay(df, sessions = 'All):
    if sessions == 'All':
        sess = df.Session.unique()
    subs = df.Subject.unique()
    df2 = pd.DataFrame(columns = ['Subject','Session','Win-stay_risk','Win-stay_optimal'] )
    for sub in subs:
        for num in sess:
            df1 = df.loc[(df.Subject == sub) & (df.Group == num)]
            if df1.empty:
                df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 'Win-stay_risk': np.nan,
                                           'Win-stay_optimal': np.nan}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
            else:
                risky_win_stay = 0
                optimal_win_stay = 0
                for i in range(1,len(df1)):
                    prev_trial = df1.iloc[i-1, :]
                    curr_trial = df1.iloc[i, :]
                    if curr_trial.risky_choice == 1:
                        if prev_trial.risky_choice == 1 and prev_trial.Rewarded == 1:
                            risky_win_stay += 1 
                    elif curr_trial.risky_choice == 0:
                        if prev_trial.risky_choice == 0 and prev_trial.Rewarded == 1:
                            optimal_win_stay += 1
                df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 'Win-stay_risk': risky_win_stay,
                                               'Win-stay_optimal': optimal_win_stay}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
    return df2

def get_lose_shift(df, subs, sess):
    df2 = pd.DataFrame(columns = ['Lose-shift_risk','Lose-shift_optimal'] )
    for sub in subs:
        for num in sess:
            df1 = df.loc[(df.Subject == sub) & (df.Group == num)]
            if df1.empty:
                df_temp = pd.DataFrame(data = {'Lose-shift_risk': np.nan,
                                           'Lose-shift_optimal': np.nan}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
            else:
                risky_lose_shift = 0
                optimal_lose_shift = 0
                for i in range(1,len(df1)):
                    prev_trial = df1.iloc[i-1, :]
                    curr_trial = df1.iloc[i, :]
                    if curr_trial.risky_choice == 0:
                        if prev_trial.risky_choice == 1 and prev_trial.Rewarded == 0:
                            risky_lose_shift += 1 
                    elif curr_trial.risky_choice == 1:
                        if prev_trial.risky_choice == 0 and prev_trial.Rewarded == 0:
                            optimal_lose_shift += 1
                df_temp = pd.DataFrame(data = {'Lose-shift_risk': risky_lose_shift,
                                               'Lose-shift_optimal': optimal_lose_shift}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
    return df2

In [10]:
sub = df.Subject.unique()
sess = df.Group.unique()
df3 = get_win_stay(df2, sub, sess)
df4 = get_lose_shift(df2, sub, sess)


<ipython-input-12-aa80143417e3>:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sessions == 'All':


UnboundLocalError: local variable 'sess' referenced before assignment

In [11]:
df5 = pd.concat([df3,df4], axis = 1)
df5.sort_values(by = ['Subject','Session'], inplace = True, ignore_index = True)
df5

,Subject,Session,Win-stay_risk,Win-stay_optimal,Lose-shift_risk,Lose-shift_optimal
0,1,55,NaN,NaN,NaN,NaN
1,1,57,NaN,NaN,NaN,NaN
2,1,59,NaN,NaN,NaN,NaN
3,1,61,NaN,NaN,NaN,NaN
4,1,63,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
121,16,63,NaN,NaN,NaN,NaN
122,16,65,NaN,NaN,NaN,NaN
123,16,69,NaN,NaN,NaN,NaN
124,16,76,NaN,NaN,NaN,NaN


In [19]:
data = [[1,1, 10, 12,13,14], [1,2, 10, 12,13,14], [1,3, 10, 12,13,14], [2,1, 1, 2,3,4],[2,2, 1, 2,3,4],[2,3, 1, 2,3,4]]
df = pd.DataFrame(data, columns=['Subject', 'Session', 'Win-stay_risk', 'Win-stay_optimal', 'Lose-shift_risk', 'Lose-shift_optimal'])
df

,Subject,Session,Win-stay_risk,Win-stay_optimal,Lose-shift_risk,Lose-shift_optimal
0,1,1,10,12,13,14
1,1,2,10,12,13,14
2,1,3,10,12,13,14
3,2,1,1,2,3,4
4,2,2,1,2,3,4
5,2,3,1,2,3,4


In [24]:
def average_sessions(df):
    df = df.drop("Session", 1)
    averages = df.groupby(['Subject']).mean()
    aver
    return averages
average_sessions(df)

<ipython-input-24-03e8df658075>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop("Session", 1)


,Win-stay_risk,Win-stay_optimal,Lose-shift_risk,Lose-shift_optimal
Subject,,,,
1,10.0,12.0,13.0,14.0
2,1.0,2.0,3.0,4.0


In [12]:
trials = df2.groupby(['Subject', 'Group'],as_index=False)['Trial'].max()
trials.loc[:,'Trial']

0      50.0
1      79.0
2      40.0
3     111.0
4      77.0
5      74.0
6      70.0
7      94.0
8      50.0
9      37.0
10     45.0
11     40.0
12     86.0
13     82.0
14     95.0
15     79.0
16     78.0
17     79.0
18     87.0
19    128.0
20    124.0
21     98.0
22    105.0
23    121.0
24     75.0
25     58.0
26     56.0
27     66.0
28     64.0
29     69.0
30     55.0
31     48.0
32     55.0
33     58.0
34     61.0
35     55.0
36     63.0
37     80.0
38     95.0
39     55.0
40     63.0
41     69.0
42     63.0
43     87.0
44     88.0
45     90.0
46     86.0
47     55.0
48     59.0
49     67.0
50     43.0
51    103.0
52     95.0
53    107.0
54     97.0
Name: Trial, dtype: float64